### Importación de librerías

In [ ]:
import os
import cv2
import numpy as np
import pickle
import mediapipe as mp
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

### Definición de variables

In [2]:
DATASET_PATH = "../../dataset/gestos_cara"
MODEL_FILE = "modelo_gestos_cara.pkl"

### Lógica de entreno

In [ ]:
MAPA_ETIQUETAS = {
    "0_Neutro": 0,
    "1_Ojos_Cerrados": 1,
    "2_Cabeza_Der": 2,
    "3_Cabeza_Izq": 3
}

# --- CONFIGURACIÓN MEDIAPIPE ---
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

# ==========================================
#           FUNCIONES AUXILIARES
# ==========================================
def normalizar_puntos(landmarks):
    """Centra y escala los puntos faciales."""
    coords = np.array([[lm.x, lm.y] for lm in landmarks])
    centroid = np.mean(coords, axis=0)
    centered = coords - centroid
    max_dist = np.max(np.abs(centered))
    return (centered / max_dist).flatten() if max_dist > 0 else centered.flatten()

def aumentar_datos(vector_original):
    """Genera variaciones sintéticas (Data Augmentation)."""
    variaciones = [vector_original]
    puntos_2d = vector_original.reshape(-1, 2)

    # 1. Ruido
    variaciones.append((puntos_2d + np.random.normal(0, 0.01, puntos_2d.shape)).flatten())

    # 2. Escalado
    variaciones.append((puntos_2d * np.random.uniform(0.95, 1.05)).flatten())
    
    # 3. Rotación
    theta = np.radians(np.random.uniform(-10, 10))
    c, s = np.cos(theta), np.sin(theta)
    matriz_rotacion = np.array(((c, -s), (s, c)))
    variaciones.append(np.dot(puntos_2d, matriz_rotacion).flatten())

    return variaciones

# ==========================================
#           PASO 1: AUDITORÍA
# ==========================================

def auditar_dataset():
    print(f"\n{'='*40}")
    print(f" PASO 1: AUDITORÍA DEL DATASET")
    print(f"{'='*40}")
    
    if not os.path.exists(DATASET_PATH):
        print(f"ERROR CRÍTICO: La ruta '{DATASET_PATH}' no existe.")
        return False

    carpetas = [d for d in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, d))]
    total_global_ok = 0
    archivos_fallidos = []

    for carpeta in carpetas:
        ruta_carpeta = os.path.join(DATASET_PATH, carpeta)
        imagenes = [f for f in os.listdir(ruta_carpeta) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        count_ok = 0
        count_fail = 0

        print(f"Carpeta '{carpeta}': {len(imagenes)} imágenes encontradas.")

        for archivo in imagenes:
            ruta_img = os.path.join(ruta_carpeta, archivo)
            img = cv2.imread(ruta_img)

            if img is None:
                count_fail += 1
                continue

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(img_rgb)

            if results.multi_face_landmarks:
                count_ok += 1
            else:
                count_fail += 1
                archivos_fallidos.append(os.path.join(carpeta, archivo))

        # Resumen por carpeta
        print(f"   -> Válidas: {count_ok} | Fallidas: {count_fail}")
        total_global_ok += count_ok

    print("-" * 30)
    if archivos_fallidos:
        print(f"Se encontraron {len(archivos_fallidos)} imágenes defectuosas (se ignorarán en el entrenamiento).")
    else:
        print("Dataset limpio.")

    # Retorna True si hay al menos una imagen válida para entrenar
    return total_global_ok > 0

# ==========================================
#            PASO 2: ENTRENAMIENTO
# ==========================================

def entrenar_modelo():
    print(f"\n{'='*40}")
    print(f" PASO 2: ENTRENAMIENTO")
    print(f"{'='*40}")

    data = []
    labels = []

    for nombre_carpeta, etiqueta_num in MAPA_ETIQUETAS.items():
        ruta_carpeta = os.path.join(DATASET_PATH, nombre_carpeta)
        if not os.path.exists(ruta_carpeta): continue
        
        archivos = os.listdir(ruta_carpeta)
        print(f"Procesando clase '{nombre_carpeta}'...")
        
        muestras_clase = 0
        for archivo in archivos:
            if not (archivo.lower().endswith(".jpg") or archivo.lower().endswith(".png")): continue
            
            img = cv2.imread(os.path.join(ruta_carpeta, archivo))
            if img is None: continue
            
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(img_rgb)
            
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    vector_base = normalizar_puntos(face_landmarks.landmark)
                    vectores_aumentados = aumentar_datos(vector_base)
                    
                    for v in vectores_aumentados:
                        data.append(v)
                        labels.append(etiqueta_num)
                        muestras_clase += 1
        
        print(f"   -> Generados {muestras_clase} vectores de características.")

    # --- ENTRENAMIENTO ---
    data = np.array(data)
    labels = np.array(labels)

    if len(data) == 0:
        print("ERROR: No se generaron datos. Revisa el dataset.")
        return

    print(f"\n Total de muestras para entrenamiento: {len(data)}")
    print(" Entrenando Red Neuronal (MLP Classifier)... espere un momento.")

    model = MLPClassifier(
        hidden_layer_sizes=(128, 64), 
        activation='relu', 
        solver='adam', 
        max_iter=1500, 
        random_state=42
    )

    # Validación cruzada rápida para ver métricas
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, data, labels, cv=kfold)
    
    print(f"Precisión estimada (Cross-Validation): {scores.mean() * 100:.2f}%")

    # Entrenamiento final con todos los datos
    model.fit(data, labels)
    
    with open(MODEL_FILE, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"\n¡ÉXITO! Modelo guardado en: {MODEL_FILE}")

# ==========================================
#           EJECUCIÓN PRINCIPAL
# ==========================================

if __name__ == "__main__":
    # 1. Ejecutar Auditoría
    dataset_valido = auditar_dataset()

    # 2. Si la auditoría es correcta, Ejecutar Entrenamiento
    if dataset_valido:
        entrenar_modelo()
    else:
        print("\n DETENIDO: No hay suficientes datos válidos para entrenar.")